In [1]:
import cv2
import numpy as np
import time
from collections import defaultdict
import math

class LaneStateMachine:
    def __init__(self, stabilityThreshold=5):
        self.state = "No Lane Detected"
        self.counters = defaultdict(int)
        self.threshold = stabilityThreshold

    def update(self, newState):
        for key in self.counters:
            if key != newState:
                self.counters[key] = 0
        self.counters[newState] += 1
        if self.counters[newState] >= self.threshold:
            if newState != self.state:
                self.state = newState
                for key in self.counters:
                    self.counters[key] = 0
        return self.state


In [3]:

def calculateSlopeAngle(x1, y1, x2, y2):
    if x2 - x1 == 0:
        return 90
    slope = (y2 - y1) / (x2 - x1)
    return math.degrees(math.atan(abs(slope)))

def detectLanes(frame, cropMarginPercent):
    height, width = frame.shape[:2]
    cropMarginX = int(width * cropMarginPercent)
    cropMarginY = int(height * 0.3)
    roi = frame[cropMarginY:, cropMarginX:width - cropMarginX]
    lab = cv2.cvtColor(roi, cv2.COLOR_BGR2LAB)
    yellowMask = cv2.inRange(lab, np.array([150, 125, 135]), np.array([255, 200, 170]))
    kernel = np.ones((5, 3), np.uint8)
    combinedMask = cv2.morphologyEx(yellowMask, cv2.MORPH_CLOSE, kernel)
    combinedMask = cv2.morphologyEx(combinedMask, cv2.MORPH_OPEN, kernel)
    maskedRoi = cv2.bitwise_and(roi, roi, mask=combinedMask)
    gray = cv2.cvtColor(maskedRoi, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 100, 150)
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=100, minLineLength=50, maxLineGap=50)
    return combinedMask, edges, lines, roi

def classifyLaneLines(lines, cropMarginPercent, height, width, prevLeft=None, prevRight=None, angleThreshold=5, slopeThreshold=0.3):
    leftLines = []
    rightLines = []
    leftAngle = None
    rightAngle = None

    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            x1 += int(cropMarginPercent * width)
            x2 += int(cropMarginPercent * width)
            y1 += int(height * 0.3)
            y2 += int(height * 0.3)
            angle = calculateSlopeAngle(x1, y1, x2, y2)
            slope = (y2 - y1) / (x2 - x1) if (x2 - x1) != 0 else float('inf')

            if 30 < angle < 90:
                if slope < 0:
                    leftLines.extend([(x1, y1), (x2, y2)])
                    leftAngle = angle
                else:
                    rightLines.extend([(x1, y1), (x2, y2)])
                    rightAngle = angle

    if prevLeft and leftAngle is not None and prevLeft['angle'] is not None:
        if abs(leftAngle - prevLeft['angle']) > angleThreshold:
            leftLines = []

    if prevRight and rightAngle is not None and prevRight['angle'] is not None:
        if abs(rightAngle - prevRight['angle']) > angleThreshold:
            rightLines = []

    if not leftLines and prevLeft:
        leftLines = prevLeft['points']

    if not rightLines and prevRight:
        rightLines = prevRight['points']

    currentLeft = {
        'points': leftLines,
        'angle': leftAngle if leftLines else (prevLeft['angle'] if prevLeft else None),
        'slope': (leftLines[1][0] - leftLines[0][0]) / (leftLines[1][1] - leftLines[0][1]) if len(leftLines) >= 2 else None
    } if leftLines else None

    currentRight = {
        'points': rightLines,
        'angle': rightAngle if rightLines else (prevRight['angle'] if prevRight else None),
        'slope': (rightLines[1][0] - rightLines[0][0]) / (rightLines[1][1] - rightLines[0][1]) if len(rightLines) >= 2 else None
    } if rightLines else None

    return leftLines, rightLines, currentLeft, currentRight

def shadeLaneRegion(frame, leftLines, rightLines):
    height = frame.shape[0]
    overlay = np.zeros_like(frame)

    if leftLines and rightLines and len(leftLines) >= 2 and len(rightLines) >= 2:
        leftPoints = np.array(leftLines).reshape(-1, 2)
        rightPoints = np.array(rightLines).reshape(-1, 2)
        leftFit = np.polyfit(leftPoints[:, 1], leftPoints[:, 0], 1)
        rightFit = np.polyfit(rightPoints[:, 1], rightPoints[:, 0], 1)
        plotY = np.linspace(height // 2, height - 1, num=height // 2)
        leftX = leftFit[0] * plotY + leftFit[1]
        rightX = rightFit[0] * plotY + rightFit[1]
        ptsLeft = np.array([np.transpose(np.vstack([leftX, plotY]))])
        ptsRight = np.array([np.flipud(np.transpose(np.vstack([rightX, plotY])))])
        pts = np.hstack((ptsLeft, ptsRight)).astype(np.int32)
        cv2.fillPoly(overlay, [pts], (0, 255, 255))

    return overlay

def identifyObstacles(frame):
    hsvFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    lowerOrange = np.array([0, 45, 50])
    upperOrange = np.array([15, 255, 255])
    lowerBlue = np.array([90, 30, 10])
    upperBlue = np.array([135, 255, 70])
    orangeMask = cv2.inRange(hsvFrame, lowerOrange, upperOrange)
    blueMask = cv2.inRange(hsvFrame, lowerBlue, upperBlue)
    obstacleMask = cv2.bitwise_or(orangeMask, blueMask)
    contours, _ = cv2.findContours(obstacleMask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    detectedObjects = []
    minContourArea = 500

    for contour in contours:
        if cv2.contourArea(contour) > minContourArea:
            x, y, w, h = cv2.boundingRect(contour)
            detectedObjects.append((x, y, w, h))
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(frame, "Object", (x, y - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)

    return detectedObjects

def main(videoPath):
    cap = cv2.VideoCapture(videoPath)

    if not cap.isOpened():
        print("Error opening video file")
        return

    laneStateMachine = LaneStateMachine(stabilityThreshold=5)
    cropMarginPercent = 0.1
    prevLeft = None
    prevRight = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (960, 540))
        height, width, _ = frame.shape
        laneMask, edges, lines, laneRoi = detectLanes(frame, cropMarginPercent)
        leftLines, rightLines, currentLeft, currentRight = classifyLaneLines(lines, cropMarginPercent, height, width, prevLeft, prevRight)

        if currentLeft:
            prevLeft = currentLeft
        if currentRight:
            prevRight = currentRight

        laneOverlay = shadeLaneRegion(frame, leftLines, rightLines)

        centerLine = None
        if leftLines and rightLines and len(leftLines) >= 2 and len(rightLines) >= 2:
            leftPoints = np.array(leftLines).reshape(-1, 2)
            rightPoints = np.array(rightLines).reshape(-1, 2)
            leftFit = np.polyfit(leftPoints[:, 1], leftPoints[:, 0], 1)
            rightFit = np.polyfit(rightPoints[:, 1], rightPoints[:, 0], 1)
            plotY = np.linspace(height // 2, height - 1, num=height // 2)
            leftX = leftFit[0] * plotY + leftFit[1]
            rightX = rightFit[0] * plotY + rightFit[1]
            centerX = (leftX + rightX) / 2
            centerLine = np.array([np.transpose(np.vstack([centerX, plotY]))]).astype(np.int32)
            cv2.polylines(laneOverlay, [centerLine], False, (0, 255, 0), 2)

        detectedBoxes = identifyObstacles(frame)

        if not leftLines and not rightLines:
            currentDecision = "No Lane Detected"
        else:
            currentDecision = "Go Straight"
            if detectedBoxes and laneOverlay.any() and centerLine is not None:
                laneAreaMask = cv2.cvtColor(laneOverlay, cv2.COLOR_BGR2GRAY)
                _, laneAreaMask = cv2.threshold(laneAreaMask, 1, 255, cv2.THRESH_BINARY)

                for (x, y, w, h) in detectedBoxes:
                    obstacleMask = np.zeros_like(laneAreaMask)
                    cv2.rectangle(obstacleMask, (x, y), (x + w, y + h), 255, -1)
                    overlap = cv2.bitwise_and(laneAreaMask, obstacleMask)

                    if cv2.countNonZero(overlap) > 0:
                        obstacleCenterX = x + w // 2
                        minDist = float('inf')
                        closestCenterX = 0

                        for point in centerLine[0]:
                            centerX, centerY = point
                            dist = abs(obstacleCenterX - centerX)
                            if dist < minDist:
                                minDist = dist
                                closestCenterX = centerX

                        if obstacleCenterX < closestCenterX:
                            currentDecision = "Turn RIGHT - Object on LEFT"
                        else:
                            currentDecision = "Turn LEFT - Object on RIGHT"
                        break

        decision = laneStateMachine.update(currentDecision)
        combinedFrame = cv2.addWeighted(frame, 0.8, laneOverlay, 0.5, 0)

        if "No Lane" in decision:
            textColor = (255, 255, 0)
        elif "STOP" in decision or "Turn" in decision:
            textColor = (0, 0, 255)
        else:
            textColor = (0, 255, 0)

        cv2.putText(combinedFrame, f"Decision: {decision}", (30, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, textColor, 2)

        for (x, y, w, h) in detectedBoxes:
            cv2.rectangle(combinedFrame, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(combinedFrame, "Object", (x, y - 8), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)  # Updated message here

        cv2.imshow("Lane and Object Detection", combinedFrame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()



if __name__ == "__main__":
    videoPath = "PXL_20250325_043754655.TS.mp4"
    main(videoPath)

leftLines :  [(np.int32(108), np.int32(513)), (np.int32(380), np.int32(241)), (np.int32(120), np.int32(538)), (np.int32(341), np.int32(292)), (np.int32(169), np.int32(451)), (np.int32(381), np.int32(239))] 
currentLeft :  {'points': [(np.int32(108), np.int32(513)), (np.int32(380), np.int32(241)), (np.int32(120), np.int32(538)), (np.int32(341), np.int32(292)), (np.int32(169), np.int32(451)), (np.int32(381), np.int32(239))], 'angle': 45.0, 'slope': np.float64(-1.0)}
leftLines :  [(np.int32(120), np.int32(500)), (np.int32(382), np.int32(238)), (np.int32(181), np.int32(470)), (np.int32(341), np.int32(292)), (np.int32(98), np.int32(524)), (np.int32(162), np.int32(460)), (np.int32(121), np.int32(538)), (np.int32(230), np.int32(417)), (np.int32(215), np.int32(404)), (np.int32(380), np.int32(239)), (np.int32(235), np.int32(409)), (np.int32(383), np.int32(244)), (np.int32(96), np.int32(525)), (np.int32(214), np.int32(407))] 
currentLeft :  {'points': [(np.int32(120), np.int32(500)), (np.int32(3

KeyboardInterrupt: 

In [3]:
import numpy as np
import cv2

def fixColor(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

def region_of_interest(img, vertices):
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, vertices, 255)
    masked_img = cv2.bitwise_and(img, mask)
    return masked_img

video_path = "PXL_20250325_043754655.TS.mp4"
cap = cv2.VideoCapture(video_path)


backSub = cv2.createBackgroundSubtractorMOG2(detectShadows=False)
imshape = (600,600)
mask_roi = np.zeros(imshape, dtype=np.uint8)
ignore_mask_color = 255

vertices = np.array([[
    (0, imshape[0]),
    (int(imshape[1] * .20), int(imshape[0] * .4)),
    (int(imshape[1] * .80), int(imshape[0] * .4)),
    (imshape[1], imshape[0])
]], dtype=np.int32)

mask_roi = cv2.fillPoly(mask_roi, vertices, ignore_mask_color)

mask = np.zeros(imshape)

vertices = np.array([
    [
        (imshape[1]*0.2,imshape[0]),
        (imshape[1] * .45, imshape[0] * .7),
        (imshape[1] * .55, imshape[0] * .7),
        (imshape[1]*0.8,imshape[0])
    ]
], dtype=np.int32)

mask_obj = cv2.fillPoly(mask, vertices, ignore_mask_color)
print(mask_obj.shape)
while True:
    ret, frame = cap.read()
    if not ret:
        print("Stream ended")
        break

    image = cv2.resize(frame, (600, 600))

    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    lower_yellow = np.array([15, 40, 60])
    upper_yellow = np.array([35, 255, 255])

    lower_white = np.array([0, 0, 200])
    upper_white = np.array([180, 5, 255])

    yellow_mask = cv2.inRange(hsv_image, lower_yellow, upper_yellow)
    white_mask = cv2.inRange(hsv_image, lower_white, upper_white)
    mask = cv2.bitwise_or(yellow_mask, white_mask)

    median = cv2.medianBlur(mask, 5)
    blur = cv2.GaussianBlur(median, (5, 5), 0)

    edges = cv2.Canny(blur, 50, 150)
    print(edges.shape)
    masked_edges = cv2.bitwise_and(edges, mask_roi)

    roi = cv2.bitwise_and(edges, edges, mask=mask_roi.astype(np.uint8))
    croppedregion = cv2.bitwise_and(image, image, mask=mask_obj.astype(np.uint8))

    fg_mask = backSub.apply(croppedregion)

    kernel = np.ones((5,5),np.uint8)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_DILATE, kernel)

    non_zero = np.count_nonzero(fg_mask)

    if non_zero > 1200:
        print("Obstacle Detected!")
    else:
        print("No Obstacle.")



    lines = cv2.HoughLinesP(roi, rho=2, theta=np.pi/180, threshold=30, minLineLength=20, maxLineGap=1)
    cv2.imshow('', fg_mask)
    mxb = np.array([[0, 0]])
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                if x2 - x1 == 0:
                    continue  # Avoid division by zero
                m = (y2 - y1) / (x2 - x1)
                b = y1 - m * x1
                mxb = np.vstack((mxb, [m, b]))


        if mxb.shape[0] > 1:
            median_right_m = np.median(mxb[mxb[:, 0] > 0, 0])
            median_left_m = np.median(mxb[mxb[:, 0] < 0, 0])
            median_right_b = np.median(mxb[mxb[:, 0] > 0, 1])
            median_left_b = np.median(mxb[mxb[:, 0] < 0, 1])

            if not np.isnan(median_right_m) and not np.isnan(median_left_m):
                x_intersect = (median_left_b - median_right_b) / (median_right_m - median_left_m)
                y_intersect = median_right_m * x_intersect + median_right_b

                left_bottom = (imshape[0] - median_left_b) / median_left_m
                right_bottom = (imshape[0] - median_right_b) / median_right_m

                line_image = np.copy(image) * 0
                cv2.line(
                    line_image,
                    (int(left_bottom), imshape[0]),
                    (int(x_intersect), int(y_intersect)),
                    (255, 0, 0), 10
                )
                cv2.line(
                    line_image,
                    (int(right_bottom), imshape[0]),
                    (int(x_intersect), int(y_intersect)),
                    (0, 0, 255), 10
                )

                lane_edges = cv2.addWeighted(image, 0.8, line_image, 1, 0)
            else:
                lane_edges = image
        else:
            lane_edges = image
    else:
        lane_edges = image
    cv2.polylines(lane_edges, [vertices], isClosed=True, color=(0, 255, 0), thickness=2)
    cv2.imshow('Road Detection', lane_edges)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

(600, 600)
(600, 600)
Obstacle Detected!
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.
(600, 600)
No Obstacle.

In [17]:
import cv2
import numpy as np
import time
from collections import defaultdict

# -------------------- Lane State Machine Class --------------------

class LaneStateMachine:
    def __init__(self, stability_threshold=5):
        self.state = "No Lane Detected"
        self.counters = defaultdict(int)
        self.threshold = stability_threshold

    def update(self, new_state):
        for key in self.counters:
            if key != new_state:
                self.counters[key] = 0
        self.counters[new_state] += 1

        if self.counters[new_state] >= self.threshold:
            if new_state != self.state:
                self.state = new_state
                for key in self.counters:
                    self.counters[key] = 0
        return self.state

# -------------------- Utility Functions --------------------

def is_dashed(lines, threshold=20):
    if len(lines) < 4:
        return False
    lines = sorted(lines, key=lambda p: p[1])
    gaps = [abs(lines[i + 2][1] - lines[i][1]) for i in range(0, len(lines) - 2, 2)]
    return sum(gaps) / len(gaps) > threshold if gaps else False

def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    cl = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8)).apply(l)
    return cv2.cvtColor(cv2.merge((cl, a, b)), cv2.COLOR_LAB2BGR)


def detect_lanes(frame, crop_margin):
    height, width = frame.shape[:2]
    roi = frame[height // 2:, crop_margin:width - crop_margin]

    # Apply CLAHE for contrast enhancement
    clahe_roi = apply_clahe(roi)

    # HSV yellow mask with tightened thresholds to reduce grass
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    # Bright yellow filtering
    hsv_yellow_mask = cv2.inRange(hsv, np.array([20, 100, 100]), np.array([35, 255, 255]))
    v_mask = cv2.inRange(v, 150, 255)
    hsv_yellow_mask = cv2.bitwise_and(hsv_yellow_mask, hsv_yellow_mask, mask=v_mask)

    # LAB yellow mask (optional but helps in poor lighting)
    lab = cv2.cvtColor(roi, cv2.COLOR_BGR2LAB)
    lab_yellow_mask = cv2.inRange(lab, np.array([150, 120, 120]), np.array([255, 160, 160]))

    # Combine HSV + LAB masks
    combined_mask = cv2.bitwise_or(hsv_yellow_mask, lab_yellow_mask)

    # Morphological filtering to suppress grass textures
    kernel = np.ones((7, 7), np.uint8)
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel)

    # Edge detection on masked bright yellow lane area
    masked_roi = cv2.bitwise_and(clahe_roi, clahe_roi, mask=combined_mask)
    gray = cv2.cvtColor(masked_roi, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 100, 200)
    edges = cv2.bitwise_and(edges, combined_mask)

    # Detect lines
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=30)
    return combined_mask, edges, lines, roi


def classify_lane_lines(lines, crop_margin, height):
    left_lines, right_lines = [], []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            x1 += crop_margin
            x2 += crop_margin
            y1 += height // 2
            y2 += height // 2

            if x2 - x1 == 0:
                continue

            slope = (y2 - y1) / (x2 - x1)
            angle = np.degrees(np.arctan(abs(slope)))

            # Only consider lines within specific angle ranges
            if 30 < angle < 80:  # Typical lane angles
                if slope < 0:
                    left_lines.extend([(x1, y1), (x2, y2)])
                else:
                    right_lines.extend([(x1, y1), (x2, y2)])
    return left_lines, right_lines

def draw_lane_lines(frame, left_lines, right_lines):
    height = frame.shape[0]
    plot_y = np.linspace(height // 2, height - 1, num=height // 2)
    line_image = np.zeros_like(frame)

    def draw_line(points, color):
        if points:
            points_np = np.array(points)
            fit = np.polyfit(points_np[:, 1], points_np[:, 0], 1)
            x_vals = fit[0] * plot_y + fit[1]
            for x, y in zip(x_vals, plot_y):
                cv2.circle(line_image, (int(x), int(y)), 2, color, -1)

    draw_line(left_lines, (255, 0, 0))
    draw_line(right_lines, (0, 0, 255))
    return line_image

def detect_obstacles(frame, obstacle_crop_margin):
    height, width = frame.shape[:2]
    hsv = cv2.cvtColor(frame[:, obstacle_crop_margin:width - obstacle_crop_margin], cv2.COLOR_BGR2HSV)

    color_ranges = [
        (np.array([0, 0, 0]), np.array([180, 255, 50]), 'Black'),
        (np.array([90, 50, 50]), np.array([130, 255, 255]), 'Blue'),
        (np.array([0, 70, 50]), np.array([10, 255, 255]), 'Red'),
        (np.array([160, 70, 50]), np.array([180, 255, 255]), 'Red')
    ]

    kernel = np.ones((5, 5), np.uint8)
    combined_mask = np.zeros_like(hsv[:, :, 0])
    color_mask_map = []

    for lower, upper, color_name in color_ranges:
        mask = cv2.inRange(hsv, lower, upper)
        color_mask_map.append((mask, color_name))
        combined_mask = cv2.bitwise_or(combined_mask, mask)

    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if 500 < area < 750:
            x, y, w, h = cv2.boundingRect(cnt)
            center_x_global = x + w // 2 + obstacle_crop_margin
            if width // 4 < center_x_global < 3 * width // 4:
                mask_roi = np.zeros_like(combined_mask)
                cv2.drawContours(mask_roi, [cnt], -1, 255, -1)
                mean_hsv = cv2.mean(hsv, mask=mask_roi)[:3]
                for lower, upper, color_name in color_ranges:
                    if np.all(lower <= mean_hsv) and np.all(mean_hsv <= upper):
                        cv2.rectangle(frame, (x + obstacle_crop_margin, y),
                                      (x + w + obstacle_crop_margin, y + h), (0, 0, 255), 2)
                        cv2.putText(frame, f"Obstacle: {color_name}",
                                    (x + obstacle_crop_margin, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                                    (0, 0, 255), 2)
                        return True
    return False

# -------------------- Main Function --------------------

def main(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file")
        return

    lane_state_machine = LaneStateMachine(stability_threshold=5)
    crop_margin = 144  # 15% of 960
    obstacle_crop_margin = 336  # 35% of 960

    prev_time, frame_count, fps = time.time(), 0, 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (960, 540))
        height, width = frame.shape[:2]

        # Lane Detection
        lane_mask, edges, lines, lane_roi = detect_lanes(frame, crop_margin)
        left_lines, right_lines = classify_lane_lines(lines, crop_margin, height)
        left_dashed = is_dashed(left_lines)
        right_dashed = is_dashed(right_lines)
        lane_overlay = draw_lane_lines(frame, left_lines, right_lines)

        # Obstacle Detection
        obstacle_detected = detect_obstacles(frame, obstacle_crop_margin)

        # Decision Making
        if obstacle_detected:
            current_decision = "STOP - Obstacle Ahead"
        elif right_dashed and not left_dashed:
            current_decision = "Right Turn Possible – Dashed Line"
        elif left_dashed and not right_dashed:
            current_decision = "Left Turn Possible – Dashed Line"
        elif not left_lines and not right_lines:
            current_decision = "No Lane Detected"
        else:
            current_decision = "Go Straight"

        decision = lane_state_machine.update(current_decision)

        # Overlay display
        combined_lower = cv2.addWeighted(lane_roi, 0.7, cv2.cvtColor(lane_mask, cv2.COLOR_GRAY2BGR), 0.3, 0)
        combined_lower = cv2.addWeighted(combined_lower, 0.9, cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR), 0.4, 0)
        full_frame = frame.copy()
        full_frame[height // 2:, crop_margin:width - crop_margin] = combined_lower
        output = cv2.addWeighted(full_frame, 1, lane_overlay, 0.6, 0)

        cv2.rectangle(output, (20, 20), (620, 80), (0, 0, 0), -1)
        cv2.putText(output, decision, (30, 65), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

        frame_count += 1
        if frame_count >= 10:
            curr_time = time.time()
            fps = frame_count / (curr_time - prev_time)
            prev_time = curr_time
            frame_count = 0

        cv2.putText(output, f"FPS: {fps:.2f}", (700, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Show windows
        cv2.imshow('Original Frame', frame)
        cv2.imshow('Lane Area', lane_overlay)
        cv2.imshow('Final Output', output)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# -------------------- Run --------------------
if __name__ == "__main__":
    main("PXL_20250325_044505516.TS.mp4")

In [1]:
import cv2
import numpy as np
import time
from collections import defaultdict

# -------------------- Lane State Machine Class --------------------
class LaneStateMachine:
    def __init__(self, stability_threshold=5):
        self.state = "No Lane Detected"
        self.counters = defaultdict(int)
        self.threshold = stability_threshold

    def update(self, new_state):
        for key in self.counters:
            if key != new_state:
                self.counters[key] = 0
        self.counters[new_state] += 1

        if self.counters[new_state] >= self.threshold:
            if new_state != self.state:
                self.state = new_state
                for key in self.counters:
                    self.counters[key] = 0
        return self.state

# -------------------- Utility Functions --------------------
def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    cl = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8)).apply(l)
    return cv2.cvtColor(cv2.merge((cl, a, b)), cv2.COLOR_LAB2BGR)

def detect_lanes(frame, crop_margin):
    height, width = frame.shape[:2]
    roi = frame[height // 2:, crop_margin:width - crop_margin]

    # Apply CLAHE for contrast enhancement
    clahe_roi = apply_clahe(roi)

    # HSV yellow mask with tightened thresholds to reduce grass
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    # Bright yellow filtering
    hsv_yellow_mask = cv2.inRange(hsv, np.array([20, 100, 100]), np.array([35, 255, 255]))
    v_mask = cv2.inRange(v, 150, 255)
    hsv_yellow_mask = cv2.bitwise_and(hsv_yellow_mask, hsv_yellow_mask, mask=v_mask)

    # LAB yellow mask (optional but helps in poor lighting)
    lab = cv2.cvtColor(roi, cv2.COLOR_BGR2LAB)
    lab_yellow_mask = cv2.inRange(lab, np.array([150, 120, 120]), np.array([255, 160, 160]))

    # Combine HSV + LAB masks
    combined_mask = cv2.bitwise_or(hsv_yellow_mask, lab_yellow_mask)

    # Morphological filtering to suppress grass textures
    kernel = np.ones((7, 7), np.uint8)
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel)

    # Edge detection on masked bright yellow lane area
    masked_roi = cv2.bitwise_and(clahe_roi, clahe_roi, mask=combined_mask)
    gray = cv2.cvtColor(masked_roi, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 100, 200)
    edges = cv2.bitwise_and(edges, combined_mask)

    # Detect lines
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=30)
    return combined_mask, edges, lines, roi

def classify_lane_lines(lines, crop_margin, height):
    left_lines, right_lines = [], []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            x1 += crop_margin
            x2 += crop_margin
            y1 += height // 2
            y2 += height // 2

            if x2 - x1 == 0:
                continue

            slope = (y2 - y1) / (x2 - x1)
            angle = np.degrees(np.arctan(abs(slope)))

            # Only consider lines within specific angle ranges
            if 30 < angle < 80:  # Typical lane angles
                if slope < 0:
                    left_lines.extend([(x1, y1), (x2, y2)])
                else:
                    right_lines.extend([(x1, y1), (x2, y2)])
    return left_lines, right_lines

def draw_lane_lines(frame, left_lines, right_lines):
    height = frame.shape[0]
    plot_y = np.linspace(height // 2, height - 1, num=height // 2)
    line_image = np.zeros_like(frame)

    def draw_line(points, color):
        if points:
            points_np = np.array(points)
            fit = np.polyfit(points_np[:, 1], points_np[:, 0], 1)
            x_vals = fit[0] * plot_y + fit[1]
            for x, y in zip(x_vals, plot_y):
                cv2.circle(line_image, (int(x), int(y)), 2, color, -1)

    draw_line(left_lines, (255, 0, 0))
    draw_line(right_lines, (0, 0, 255))
    return line_image

# -------------------- Object Detection within Shaded Region --------------------

def detect_objects_in_shaded_region(road_mask, frame):
    # Find contours in the road mask (Shaded area)
    contours, _ = cv2.findContours(road_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter out small contours (noise)
    large_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 1000]  # Adjust the area threshold

    # Get road boundaries
    height, width = frame.shape[:2]
    road_y_start = height // 2
    road_y_end = height

    # Draw bounding boxes around detected objects, constrained within the road area
    result_frame = frame.copy()
    for cnt in large_contours:
        x, y, w, h = cv2.boundingRect(cnt)

        # Ensure the bounding box is constrained within the road area
        if y + h <= road_y_end and y >= road_y_start:
            # Adjust bounding box to be centered horizontally and placed at the bottom
            center_x = (frame.shape[1] // 2) - (w // 2)  # Centered horizontally
            center_y = road_y_end - h  # Placed at the bottom vertically

            # Draw the bounding box at the shifted coordinates
            cv2.rectangle(result_frame, (center_x, center_y), (center_x + w, center_y + h), (0, 255, 0), 2)  # Green bounding boxes

    return result_frame, large_contours

def main(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file")
        return

    lane_state_machine = LaneStateMachine(stability_threshold=5)
    crop_margin = 144  # 15% of 960
    obstacle_crop_margin = 336  # 35% of 960

    prev_time, frame_count, fps = time.time(), 0, 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (960, 540))
        height, width = frame.shape[:2]

        # Lane Detection
        lane_mask, edges, lines, lane_roi = detect_lanes(frame, crop_margin)
        left_lines, right_lines = classify_lane_lines(lines, crop_margin, height)
        left_dashed = is_dashed(left_lines)
        right_dashed = is_dashed(right_lines)
        lane_overlay = draw_lane_lines(frame, left_lines, right_lines)

        # Object Detection within Shaded Yellow Region (Road)
        detected_frame, large_contours = detect_objects_in_shaded_region(lane_mask, frame)

        # Decision Making
        if large_contours:
            current_decision = "STOP - Obstacle Ahead"
        elif right_dashed and not left_dashed:
            current_decision = "Right Turn Possible – Dashed Line"
        elif left_dashed and not right_dashed:
            current_decision = "Left Turn Possible – Dashed Line"
        elif not left_lines and not right_lines:
            current_decision = "No Lane Detected"
        else:
            current_decision = "Go Straight"

        decision = lane_state_machine.update(current_decision)

        # Overlay display
        combined_lower = cv2.addWeighted(lane_roi, 0.7, cv2.cvtColor(lane_mask, cv2.COLOR_GRAY2BGR), 0.3, 0)
        combined_lower = cv2.addWeighted(combined_lower, 0.9, cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR), 0.4, 0)
        full_frame = frame.copy()
        full_frame[height // 2:, crop_margin:width - crop_margin] = combined_lower
        output = cv2.addWeighted(full_frame, 1, lane_overlay, 0.6, 0)

        cv2.rectangle(output, (20, 20), (620, 80), (0, 0, 0), -1)
        cv2.putText(output, decision, (30, 65), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

        frame_count += 1
        if frame_count >= 10:
            curr_time = time.time()
            fps = frame_count / (curr_time - prev_time)
            prev_time = curr_time
            frame_count = 0

        cv2.putText(output, f"FPS: {fps:.2f}", (700, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Show windows
        cv2.imshow('Original Frame', frame)
        cv2.imshow('Lane Area', lane_overlay)
        cv2.imshow('Final Output', output)
        cv2.imshow("Detected Objects", detected_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main("PXL_20250325_043754655.TS.mp4")  # Update the video path


NameError: name 'is_dashed' is not defined

In [8]:
import cv2
import numpy as np

def detect_obstacles(frame, obstacle_crop_margin):
    height, width = frame.shape[:2]
    hsv = cv2.cvtColor(frame[:, obstacle_crop_margin:width - obstacle_crop_margin], cv2.COLOR_BGR2HSV)

    color_ranges = [
        (np.array([0, 0, 0]), np.array([180, 255, 50]), 'Black'),
        (np.array([90, 50, 50]), np.array([130, 255, 255]), 'Blue'),
        (np.array([0, 70, 50]), np.array([10, 255, 255]), 'Red'),
        (np.array([160, 70, 50]), np.array([180, 255, 255]), 'Red')
    ]

    kernel = np.ones((5, 5), np.uint8)
    combined_mask = np.zeros_like(hsv[:, :, 0])
    color_mask_map = []

    for lower, upper, color_name in color_ranges:
        mask = cv2.inRange(hsv, lower, upper)
        color_mask_map.append((mask, color_name))
        combined_mask = cv2.bitwise_or(combined_mask, mask)

    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for cnt in contours:
        area = cv2.contourArea(cnt)
        if 500 < area < 750:
            x, y, w, h = cv2.boundingRect(cnt)
            center_x_global = x + w // 2 + obstacle_crop_margin
            if width // 4 < center_x_global < 3 * width // 4:
                mask_roi = np.zeros_like(combined_mask)
                cv2.drawContours(mask_roi, [cnt], -1, 255, -1)
                mean_hsv = cv2.mean(hsv, mask=mask_roi)[:3]
                for lower, upper, color_name in color_ranges:
                    if np.all(lower <= mean_hsv) and np.all(mean_hsv <= upper):
                        cv2.rectangle(frame, (x + obstacle_crop_margin, y),
                                      (x + w + obstacle_crop_margin, y + h), (0, 0, 255), 2)
                        cv2.putText(frame, f"Obstacle: {color_name}",
                                    (x + obstacle_crop_margin, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6,
                                    (0, 0, 255), 2)
                        return True
    return False

def band_pass_thresholding_and_detection(image, lower_bound, upper_bound, min_area=1000):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, lower_mask = cv2.threshold(gray, lower_bound[0], upper_bound[0], cv2.THRESH_BINARY)
    _, upper_mask = cv2.threshold(gray, lower_bound[1], upper_bound[1], cv2.THRESH_BINARY_INV)
    road_mask = cv2.bitwise_and(lower_mask, upper_mask)
    contours, _ = cv2.findContours(road_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    large_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]
    result_image = image.copy()
    for cnt in large_contours:
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(result_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    return result_image, road_mask

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file {video_path}")
        return

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_resized = cv2.resize(frame, (960, 540))

        # Apply the band-pass thresholding
        processed_frame, road_mask = band_pass_thresholding_and_detection(frame_resized, lower_bound=(120, 130), upper_bound=(200, 210))

        # Apply the obstacle detection
        obstacle_detected = detect_obstacles(frame_resized, obstacle_crop_margin=336)
        decision = "Go Straight"  # Default decision
        if obstacle_detected:
            decision = "STOP - Obstacle Ahead"

        # Display decision on frame
        cv2.rectangle(processed_frame, (20, 20), (620, 80), (0, 0, 0), -1)
        cv2.putText(processed_frame, decision, (30, 65), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

        # Show the processed frame with decision
        cv2.imshow('Processed Video', processed_frame)

        # Exit condition
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

process_video("PXL_20250325_043754655.TS.mp4")

In [16]:
import cv2
import numpy as np
import time
from collections import defaultdict

# -------------------- Lane State Machine Class --------------------

class LaneStateMachine:
    def __init__(self, stability_threshold=5):
        self.state = "No Lane Detected"
        self.counters = defaultdict(int)
        self.threshold = stability_threshold

    def update(self, new_state):
        for key in self.counters:
            if key != new_state:
                self.counters[key] = 0
        self.counters[new_state] += 1

        if self.counters[new_state] >= self.threshold:
            if new_state != self.state:
                self.state = new_state
                for key in self.counters:
                    self.counters[key] = 0
        return self.state

# -------------------- Utility Functions --------------------

def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    cl = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8)).apply(l)
    return cv2.cvtColor(cv2.merge((cl, a, b)), cv2.COLOR_LAB2BGR)

def detect_lanes(frame, crop_margin):
    height, width = frame.shape[:2]
    roi = frame[height // 2:, crop_margin:width - crop_margin]

    # Apply CLAHE for contrast enhancement
    clahe_roi = apply_clahe(roi)

    # HSV yellow mask with tightened thresholds to reduce grass
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    # Bright yellow filtering
    hsv_yellow_mask = cv2.inRange(hsv, np.array([20, 100, 100]), np.array([35, 255, 255]))
    v_mask = cv2.inRange(v, 150, 255)
    hsv_yellow_mask = cv2.bitwise_and(hsv_yellow_mask, hsv_yellow_mask, mask=v_mask)

    # LAB yellow mask (optional but helps in poor lighting)
    lab = cv2.cvtColor(roi, cv2.COLOR_BGR2LAB)
    lab_yellow_mask = cv2.inRange(lab, np.array([150, 120, 120]), np.array([255, 160, 160]))

    # Combine HSV + LAB masks
    combined_mask = cv2.bitwise_or(hsv_yellow_mask, lab_yellow_mask)

    # Morphological filtering to suppress grass textures
    kernel = np.ones((7, 7), np.uint8)
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel)

    # Edge detection on masked bright yellow lane area
    masked_roi = cv2.bitwise_and(clahe_roi, clahe_roi, mask=combined_mask)
    gray = cv2.cvtColor(masked_roi, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 100, 200)
    edges = cv2.bitwise_and(edges, combined_mask)

    # Detect lines
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=30)
    return combined_mask, edges, lines, roi

def classify_lane_lines(lines, crop_margin, height):
    left_lines, right_lines = [], []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            x1 += crop_margin
            x2 += crop_margin
            y1 += height // 2
            y2 += height // 2

            if x2 - x1 == 0:
                continue

            slope = (y2 - y1) / (x2 - x1)
            angle = np.degrees(np.arctan(abs(slope)))

            # Only consider lines within specific angle ranges
            if 30 < angle < 80:  # Typical lane angles
                if slope < 0:
                    left_lines.extend([(x1, y1), (x2, y2)])
                else:
                    right_lines.extend([(x1, y1), (x2, y2)])
    return left_lines, right_lines

def draw_lane_lines(frame, left_lines, right_lines):
    height = frame.shape[0]
    plot_y = np.linspace(height // 2, height - 1, num=height // 2)
    line_image = np.zeros_like(frame)

    def draw_line(points, color):
        if points:
            points_np = np.array(points)
            fit = np.polyfit(points_np[:, 1], points_np[:, 0], 1)
            x_vals = fit[0] * plot_y + fit[1]
            for x, y in zip(x_vals, plot_y):
                cv2.circle(line_image, (int(x), int(y)), 2, color, -1)

    draw_line(left_lines, (255, 0, 0))
    draw_line(right_lines, (0, 0, 255))
    return line_image

# Define the improved road mask and object detection function
def road_mask_and_detect(frame, lower_bound=(120, 130), upper_bound=(200, 210), min_area=1000):
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply the road mask based on thresholds
    _, lower_mask = cv2.threshold(gray, lower_bound[0], upper_bound[0], cv2.THRESH_BINARY)
    _, upper_mask = cv2.threshold(gray, lower_bound[1], upper_bound[1], cv2.THRESH_BINARY_INV)
    road_mask = cv2.bitwise_and(lower_mask, upper_mask)

    # Find contours in the road mask
    contours, _ = cv2.findContours(road_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter based on size (min_area)
    large_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_area]

    # Draw bounding boxes around detected obstacles on the road
    result_image = frame.copy()
    for cnt in large_contours:
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(result_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    return result_image, road_mask

# -------------------- Main Function --------------------

def main(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file")
        return

    lane_state_machine = LaneStateMachine(stability_threshold=5)
    crop_margin = 144  # 15% of 960
    obstacle_crop_margin = 336  # 35% of 960

    prev_time, frame_count, fps = time.time(), 0, 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (960, 540))
        height, width = frame.shape[:2]

        # Lane Detection
        lane_mask, edges, lines, lane_roi = detect_lanes(frame, crop_margin)
        left_lines, right_lines = classify_lane_lines(lines, crop_margin, height)
        left_dashed = is_dashed(left_lines)
        right_dashed = is_dashed(right_lines)
        lane_overlay = draw_lane_lines(frame, left_lines, right_lines)

        # Apply road mask and detect objects
        processed_frame, road_mask = road_mask_and_detect(frame)

        # Decision Making
        decision = "Go Straight"  # Default decision
        obstacle_detected = False  # Placeholder; apply your detection condition here

        if obstacle_detected:
            decision = "STOP - Obstacle Ahead"
        elif right_dashed and not left_dashed:
            decision = "Right Turn Possible – Dashed Line"
        elif left_dashed and not right_dashed:
            decision = "Left Turn Possible – Dashed Line"
        elif not left_lines and not right_lines:
            decision = "No Lane Detected"

        # Overlay display
        cv2.rectangle(processed_frame, (20, 20), (620, 80), (0, 0, 0), -1)  # Background for text
        cv2.putText(processed_frame, decision, (30, 65), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

        # Show the final video output
        cv2.imshow('Processed Video', processed_frame)

        # Stop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# -------------------- Run --------------------
if __name__ == "__main__":
    main("PXL_20250325_043754655.TS.mp4")  # Adjust path as needed


In [13]:
import cv2
import numpy as np
import time
from collections import defaultdict

# -------------------- Lane State Machine Class --------------------

class LaneStateMachine:
    def __init__(self, stability_threshold=5):
        self.state = "No Lane Detected"
        self.counters = defaultdict(int)
        self.threshold = stability_threshold

    def update(self, new_state):
        for key in self.counters:
            if key != new_state:
                self.counters[key] = 0
        self.counters[new_state] += 1

        if self.counters[new_state] >= self.threshold:
            if new_state != self.state:
                self.state = new_state
                for key in self.counters:
                    self.counters[key] = 0
        return self.state

# -------------------- Utility Functions --------------------

def apply_clahe(image):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    cl = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8)).apply(l)
    return cv2.cvtColor(cv2.merge((cl, a, b)), cv2.COLOR_LAB2BGR)

def detect_lanes(frame, crop_margin_percent):
    height, width = frame.shape[:2]
    crop_margin_x = int(width * crop_margin_percent)
    crop_margin_y = int(height * 0.5)  # Half of the height for the ROI

    roi = frame[crop_margin_y:, crop_margin_x:width - crop_margin_x]

    # Apply CLAHE for contrast enhancement
    clahe_roi = apply_clahe(roi)

    # HSV yellow mask with tightened thresholds to reduce grass
    hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
    h, s, v = cv2.split(hsv)

    # Bright yellow filtering
    hsv_yellow_mask = cv2.inRange(hsv, np.array([20, 100, 100]), np.array([35, 255, 255]))
    v_mask = cv2.inRange(v, 150, 255)
    hsv_yellow_mask = cv2.bitwise_and(hsv_yellow_mask, hsv_yellow_mask, mask=v_mask)

    # LAB yellow mask
    lab = cv2.cvtColor(roi, cv2.COLOR_BGR2LAB)
    lab_yellow_mask = cv2.inRange(lab, np.array([150, 120, 120]), np.array([255, 160, 160]))

    # Combine HSV + LAB masks
    combined_mask = cv2.bitwise_or(hsv_yellow_mask, lab_yellow_mask)

    # Morphological filtering
    kernel = np.ones((7, 7), np.uint8)
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_CLOSE, kernel)
    combined_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel)

    # Edge detection
    masked_roi = cv2.bitwise_and(clahe_roi, clahe_roi, mask=combined_mask)
    gray = cv2.cvtColor(masked_roi, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 100, 200)
    edges = cv2.bitwise_and(edges, combined_mask)

    # Detect lines
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=30)
    return combined_mask, edges, lines, roi

def classify_lane_lines(lines, crop_margin_percent, height):
    left_lines, right_lines = [], []
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            x1 += int(crop_margin_percent * 960)
            x2 += int(crop_margin_percent * 960)
            y1 += height // 2
            y2 += height // 2

            if x2 - x1 == 0:
                continue

            slope = (y2 - y1) / (x2 - x1)
            angle = np.degrees(np.arctan(abs(slope)))

            if 30 < angle < 80:  # Typical lane angles
                if slope < 0:
                    left_lines.extend([(x1, y1), (x2, y2)])
                else:
                    right_lines.extend([(x1, y1), (x2, y2)])
    return left_lines, right_lines

def shade_lane_region(frame, left_lines, right_lines):
    height = frame.shape[0]
    overlay = np.zeros_like(frame)

    if left_lines and right_lines:
        left_points = np.array(left_lines).reshape(-1, 2)
        right_points = np.array(right_lines).reshape(-1, 2)

        left_fit = np.polyfit(left_points[:, 1], left_points[:, 0], 1)
        right_fit = np.polyfit(right_points[:, 1], right_points[:, 0], 1)

        plot_y = np.linspace(height // 2, height - 1, num=height // 2)
        left_x = left_fit[0] * plot_y + left_fit[1]
        right_x = right_fit[0] * plot_y + right_fit[1]

        pts_left = np.array([np.transpose(np.vstack([left_x, plot_y]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_x, plot_y])))])
        pts = np.hstack((pts_left, pts_right)).astype(np.int32)

        cv2.fillPoly(overlay, [pts], (0, 255, 255))  # Yellow fill

    return overlay

# -------------------- Main Function --------------------

def main(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file")
        return

    lane_state_machine = LaneStateMachine(stability_threshold=5)
    crop_margin_percent = 0.2 # 15% of width for left/right cropping
    obstacle_crop_margin_percent = 0.35  # 35% for obstacle detection cropping

    prev_time, frame_count, fps = time.time(), 0, 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (960, 540))
        height, width = frame.shape[:2]

        # Lane Detection
        lane_mask, edges, lines, lane_roi = detect_lanes(frame, crop_margin_percent)
        left_lines, right_lines = classify_lane_lines(lines, crop_margin_percent, height)
        lane_overlay = shade_lane_region(frame, left_lines, right_lines)

        # Obstacle Detection (if required)
        obstacle_detected = False  # Remove this function if not needed

        # Decision Making
        if obstacle_detected:
            current_decision = "STOP - Obstacle Ahead"
        elif not left_lines and not right_lines:
            current_decision = "No Lane Detected"
        else:
            current_decision = "Go Straight"

        decision = lane_state_machine.update(current_decision)

        # Overlay display
        combined_lower = cv2.addWeighted(lane_roi, 0.7, cv2.cvtColor(lane_mask, cv2.COLOR_GRAY2BGR), 0.3, 0)
        combined_lower = cv2.addWeighted(combined_lower, 0.9, cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR), 0.4, 0)
        full_frame = frame.copy()
        full_frame[height // 2:, int(width * crop_margin_percent):width - int(width * crop_margin_percent)] = combined_lower
        output = cv2.addWeighted(full_frame, 1, lane_overlay, 0.6, 0)
        cv2.rectangle(output, (20, 20), (620, 80), (0, 0, 0), -1)
        cv2.putText(output, decision, (30, 65), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)

        frame_count += 1
        if frame_count >= 10:
            curr_time = time.time()
            fps = frame_count / (curr_time - prev_time)
            prev_time = curr_time
            frame_count = 0

        cv2.putText(output, f"FPS: {fps:.2f}", (700, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # Show windows
        cv2.imshow('Original Frame', frame)
        cv2.imshow('Lane Area', lane_overlay)
        cv2.imshow('Final Output', output)
        cv2.imshow("Canny Edge Output",edges)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = "PXL_20250325_043754655.TS.mp4"  # Update the video path
    main(video_path)

In [15]:
import cv2
import numpy as np

# Define the function to create the mask for the road area
def create_road_mask(frame, lower_bound=(120, 130), upper_bound=(200, 210)):
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply the band-pass thresholding approach
    _, lower_mask = cv2.threshold(gray, lower_bound[0], upper_bound[0], cv2.THRESH_BINARY)
    _, upper_mask = cv2.threshold(gray, lower_bound[1], upper_bound[1], cv2.THRESH_BINARY_INV)

    # Create the road mask
    road_mask = cv2.bitwise_and(lower_mask, upper_mask)

    return road_mask

# Function to perform Connected Component Analysis (CCA) to detect objects in the road mask
def detect_objects_in_road_mask(road_mask):
    # Find contours from the road mask (Connected Component Analysis)
    contours, _ = cv2.findContours(road_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter the contours based on the area (to ignore noise)
    large_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 1000]  # Adjust threshold as needed

    return large_contours

# Function to apply CCA, detect objects, and draw bounding boxes
def process_frame_and_detect_objects(frame):
    # Create the road mask
    road_mask = create_road_mask(frame)

    # Perform CCA to detect objects in the road region
    large_contours = detect_objects_in_road_mask(road_mask)

    # Draw bounding boxes around detected objects
    result_frame = frame.copy()
    for cnt in large_contours:
        x, y, w, h = cv2.boundingRect(cnt)
        cv2.rectangle(result_frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green bounding boxes

    return result_frame, road_mask

# Open the video and process it
video_path = "PXL_20250325_043754655.TS.mp4"  # Replace with your video path
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error opening video file")
else:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Resize frame for processing
        frame_resized = cv2.resize(frame, (960, 540))

        # Process the frame and detect objects in the road region
        processed_frame, road_mask = process_frame_and_detect_objects(frame_resized)

        # Display the processed frame with bounding boxes
        cv2.imshow('Processed Frame', processed_frame)

        # Exit if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the video capture and close windows
cap.release()
cv2.destroyAllWindows()
